In [1]:
import ast
import json
import nltk
import pandas as pd

#nltk.download('punkt')

with open('data/recipes_raw_nosource_ar.json', 'r') as f:
    recipes1 = json.load(f)

In [7]:
class Recipe:
    def __init__(self, uid, r):
        self.measures = ['bunch', 'block', 'cup', 'cups', 'pinch', 'tablespoon', 'tablespoons', 'ounces', 'ounce', 'oz',
                         'teaspoon', 'teaspoons', 'tsp', 'pound', 'pounds', 'sheets', 'slice', 'wheel', 'bag', 'bags',
                         'gram', 'grams', 'sticks', 'ml', 'mL', 'kg']
        self.uid = uid
        self.shopping_list = []
        self.instructions = r['instructions']
        self.ingredients = {x: {} for x in r['ingredients']}
        if 'title' in r:
            self.title = r['title']
            
    # strip and lower

    def ingredients_list(self):
        for entry in self.ingredients:
            clean_entry = entry.replace('-', ' ').replace('/', ' ').lower()
            tokens = nltk.word_tokenize(clean_entry)
            pos = nltk.pos_tag(tokens)
            #print(pos)
            unit = [x[0] for x in pos if x in self.measures]
            qty = [x[0] for x in pos if x[1] == 'CD' or x[1] == 'NNP']
            item = [x[0] for x in pos if x[1] == 'NN' and x[0] not in self.measures or x[1] =='NNS' and x[0] not in self.measures]
            notes = [x for x in tokens if x not in qty and x not in item]
            self.ingredients[entry]['qty'] = ' '.join(qty)
            self.ingredients[entry]['ingredient'] = ' '.join(item)
            self.ingredients[entry]['notes'] = ' '.join(notes)
            self.shopping_list.append(' '.join(item))
            

In [8]:
class CookBook:
    def __init__(self, recipes):
        self.recipes = recipes
        self.ingredients = []
        self.invalid = 0

        
    def get_ingredients(self):
        for r in self.recipes.keys():
            if 'ingredients' not in self.recipes[r] or 'instructions' not in self.recipes[r]:
                self.invalid += 1
                continue
            else:
                rec = Recipe(r, self.recipes[r])
                rec.ingredients_list()
                self.ingredients = self.ingredients  + rec.shopping_list
        print("INVALID COUNT: ", self.invalid)
        return self.ingredients

    #def to_dict(self):
        

In [9]:

cb = CookBook(recipes)
shopping_list = pd.Series(cb.get_ingredients()).unique()
print(shopping_list)

INVALID COUNT:  280
['skinless boneless chicken breast halves advertisement' 'advertisement'
 'cans cream chicken soup advertisement' ...
 'tomatoes jalapeno peppers advertisement'
 'panang style curry paste advertisement' 'spelt flakes advertisement']


In [13]:
from copy import deepcopy
s = [ item.replace(' advertisement', '') for item in shopping_list]
print(len(s))
for i in range(2050, 2100):
    print(s[i])

17993
slices juice
box round wooden toothpicks
package pistachio
medium onions cubes
carrots fourths lengthwise strips
sodium
sun paste
pineapple rings
package ladyfingers
coffee liqueur
pastry
brown potatoes
barbeque seasoning
spinach thawed
mangos
chipotle powder
fluid jigger bourbon whiskey
corn chex® cereal
rice chex® cereal
wheat chex® cereal
bite size pretzels
flavor bite size chips size bagel chips inch pieces
slices prosciutto
brazil nuts
pork tenderloin medallions
yukon gold potatoes fries
milk mozzarella
barbecue
beer room temperature
corn crumbs
slices farmhouse style bread cubes
day bread size pieces
medium ripe tomatoes wedges
mozzarella size pieces
cocktail
oranges
inch thick slices day bread
package sausage inch pieces
kiwi
cheese basil sun tomatoes
head leaf lettuce
pods inch pieces
oil cooking spray
asparagus ends
habanero peppers
tomatoes chiles ro*tel
mccormick® mild chili mix
package cream cheese room temperature
fluid bottle beer
bunches inch pieces


In [150]:
fish = ['anchovies', 'barracuda', 'basa', 'bass', 'cod', 'sablefish', 'blowfish', 'bluefish',
        'bombay duck', 'bream', 'brill', 'butter fish', 'catfish', 'dogfish', 'dorade', 'eel',
        'flounder', 'grouper', 'haddock', 'hake', 'halibut', 'herring', 'ilish', 'john dory', 'lamprey',
        'lingcod', 'mackerel', 'mahi mahi', 'monkfish', 'mullet', 'orange roughy', 'parrotfish', 'patagonian toothfish',
        'perch', 'pike', 'pilchard', 'pollock', 'pomfret', 'pompano', 'sablefish', 'salmon', 'sanddab', 'sardine',
        'sea bass', 'shad', 'alewife', 'shark', 'skate', 'smelt', 'snakehead', 'snapper', 'rockfish', 'sole', 'sprat', 
        'sturgeon', 'surimi', 'swordfish', 'tilapia', 'tilefish', 'trout', 'tuna', 'albacore', 'yellowfin', 'bigeye',
        'bluefin', 'dogtooth', 'turbot', 'wahoo', 'whitefish', 'whiting', 'witch', 'righteye', 'whitebait', 'roe', 'caviar',
        'ikura', 'kazunoko', 'lumpfish','masago', 'capelin', 'tobiko', 'flying-fish', 'shellfish', 'crustaceans', 'crab',
        'dungeness', 'crayfish', 'langostino', 'lobster', 'shrimp', 'prawns', 'mollusca', 'cockle', 'cuttlefish', 'clam',
        'loco', 'mussel', 'octopus', 'oyster', 'periwinkle', 'scallop', 'squid', 'escargot', 'snails', 'nautilus', 'dolphin',
        'seal', 'whale', 'echinoderms', 'sea cucumber', 'uni', 'sea urchin', 'roe', 'medusozoa', 'Tunicates', 'sea squirts',
        'microcosmus', 'sabatieri', 'sea fig', 'sea violet', 'crabmeat']

meat = ['chicken', 'turkey', 'duck', 'mallard', 'goose','greylag', 'guinea', 'fowl', 'pigeon', 'dove', 'quail',
        'partridge', 'swiftlet', 'swan', 'beef', 'steak', 'sirloin', 'veal', 'brisket', 'pork', 'ham', 'bacon', 'shank', 'lamb',
        'mutton', 'goat', 'chuck']

keyword = ['meat', 'seafood', 'vegetarian', 'vegan']


In [15]:
with open('data/train.json/train.json', 'r') as f:
    nationalities = json.load(f)

In [16]:
class DietModel:
    def __init__(self, training, ratios):
        self.training = training
        self.model = {c: {'ingredients': {},
                          'total_items': 0,
                          'ratio': ratios[c]['ratio'] * 100000} for c in ratios}
        self.model['world'] = {'ingredients': {},
                               'total_items': 0,
                               'ratio': 100000}
        
    def build_model(self):
        for item in self.training:
            if item['cuisine'] not in self.model:
                self.model[item['cuisine']] = {}
            for i in item['ingredients']:
                if i not in self.model['world']['ingredients']:
                    self.model['world']['ingredients'][i] = 1
                    self.model['world']['total_items'] += 1
                else:
                    self.model['world']['ingredients'][i] +=1
                if i not in self.model[item['cuisine']]['ingredients']:
                    self.model[item['cuisine']]['ingredients'][i] = 1
                    self.model[item['cuisine']]['total_items'] += 1
                else:
                    self.model[item['cuisine']]['ingredients'][i] +=1
                    
    def clean_up(self):
        for region in self.model:
            clean = []
            for item in self.model[region]['ingredients']:
                if self.model[region]['ingredients'][item] <= 2:
                    clean.append(item)
            for item in clean:
                del self.model[region]['ingredients'][item]
                self.model[region]['total_items'] -=1
                    
    

In [17]:
print(len(nationalities))

countries = {}

for item in nationalities:
    if item['cuisine'] not in countries:
        countries[item['cuisine']] = {'count':1}
    else:
        countries[item['cuisine']]['count'] += 1

sum = 0

for country in countries:
    countries[country]['ratio'] = countries[country]['count']/39774 
    sum+= countries[country]['ratio']
    print(country, ": ", countries[country])
    
    

39774
greek :  {'count': 1175, 'ratio': 0.02954191180167949}
southern_us :  {'count': 4320, 'ratio': 0.10861366721979182}
filipino :  {'count': 755, 'ratio': 0.018982249710866397}
indian :  {'count': 3003, 'ratio': 0.07550158394931362}
jamaican :  {'count': 526, 'ratio': 0.013224719666113542}
spanish :  {'count': 989, 'ratio': 0.024865490018605118}
italian :  {'count': 7838, 'ratio': 0.19706340825665009}
mexican :  {'count': 6438, 'ratio': 0.16186453462060643}
chinese :  {'count': 2673, 'ratio': 0.06720470659224619}
british :  {'count': 804, 'ratio': 0.020214210288127924}
thai :  {'count': 1539, 'ratio': 0.03869361894705084}
vietnamese :  {'count': 825, 'ratio': 0.020742193392668577}
cajun_creole :  {'count': 1546, 'ratio': 0.038869613315231054}
brazilian :  {'count': 467, 'ratio': 0.011741338562880274}
french :  {'count': 2646, 'ratio': 0.0665258711721225}
japanese :  {'count': 1423, 'ratio': 0.03577714084577865}
irish :  {'count': 667, 'ratio': 0.01676974908231508}
korean :  {'count'

In [18]:
food_model = DietModel(nationalities, countries)
food_model.build_model()
food_model.clean_up()

for c in food_model.model:
    print('-----------------------------')
    print(c, ": ")
    for ingredient in food_model.model[c]:
        print(ingredient, ": ", food_model.model[c][ingredient])
        


-----------------------------
greek : 
ingredients :  {'romaine lettuce': 39, 'black olives': 31, 'grape tomatoes': 26, 'garlic': 216, 'pepper': 203, 'purple onion': 186, 'seasoning': 3, 'garbanzo beans': 23, 'feta cheese crumbles': 252, 'ground pork': 4, 'finely chopped fresh parsley': 8, 'onions': 185, 'salt': 572, 'vinegar': 3, 'minced garlic': 68, 'dried oregano': 267, 'red wine vinegar': 99, 'olive oil': 504, 'lemon juice': 183, 'orange': 12, 'anise': 5, 'cinnamon sticks': 31, 'orange blossom honey': 3, 'sugar': 77, 'lemon': 129, 'clove': 14, 'honey': 67, 'plain whole-milk yogurt': 5, 'fresh dill': 108, 'yoghurt': 10, 'large eggs': 72, 'cheese': 15, 'feta cheese': 191, 'phyllo': 12, 'kefalotyri': 9, 'ground black pepper': 221, 'extra-virgin olive oil': 229, 'hamburger buns': 6, 'paprika': 20, 'chopped fresh mint': 68, 'ground cinnamon': 73, 'balsamic vinegar': 24, 'baby spinach leaves': 9, 'ground lamb': 68, 'dried mint flakes': 15, 'tomatoes': 156, 'dried dillweed': 10, 'red wine

In [19]:
count = 0
for food in s:
    guess = 0
    #food = food.split()
    #for f in food:
    if food in food_model.model['world']['ingredients']:
        guess +=1
    if guess == 0:
        print(food)
        count +=1
print(count)

skinless boneless chicken breast halves
advertisement
cans cream chicken soup
onion
packages biscuit dough pieces
cans cream mushroom soup
package onion soup mix
wheat rotini pasta
medium onion
cloves
italian crumbs
reynolds aluminum foil
potato crust
russet potatoes chunks
salt ground pepper
rest
powder
egg
sauce oelek
oyster
shallot
dungeness crabs
onion part
ground
shoulder inch pieces
onions size pieces
sausage
paste
cans sauce
bouillon granules
inch crusts
spray
brownie mix hines®
cream liqueur baileys®
pastry inch pie
smith apples
head cabbage
half half cream
salt pepper
pinches
bottle barbeque
slices
cornstarch
clove garlic
skinless chicken
avocados
roma plum tomatoes
ground beef chuck
bulk sausage
beans spicy sauce
cans tomatoes juice
stalks celery
chile peppers
cubes beef bouillon
pepper sauce tabasco™
package cheddar cheese
style crumbs
cream mushroom soup
kernel corn
fluid beer
cans tomatoes chilies
package seasoning
skinless boneless chicken
ziti pasta
medium bananas
cream 

chicken broth splash
ham cubes
foil
package basil
package mozzarella balls
beef sirloin beef round steaks thick
head cabbage choy
medium peppers
medium onions pieces
package crust pie pastry
stone ground cornmeal
jar bacon bits
fluid limeade concentrate
fluid lemonade concentrate
milliliter bottle rum
bran cereal
butternut squash inch pieces
packet sazon
jars pasta sauce
sofrito sauce
tips
wheat crackers
fluid bottle tomato juice cocktail
raspberry
boil lasagna noodles
chicken tenders strips
bell pepper inch strips
bell peppers inch strips
boneless beef loin
linguini pasta
beef sirloin size slices
pasta package directions
gluten vanilla extract
chicken shreds chunks
corn tortillas quarters
gallon strawberry sherbet
fluid punch
package bite size potato nuggets tater tots®
style steak seasoning
round bread
potato rolls
package instant potatoes
crescent dinner rolls
tiger
andouille sausage rounds
package boil lasagna noodles
squid rings
cubed
goat gouda
rotisserie chicken meat chunks
onio

paneer inch cubes
purple sugar
yellow sugar
muffin split
boneless pork loin chops inch thick
portabella mushroom caps inch diameter stems
container cream spread onion chives
chives garnish
inch thick boneless skin center cut fillets
baby clams
jar cream
chayote squashes inch pieces
williamsburg butter
salsa choice hotness
liter ginger soda
buttermilk dressing
lengths
breast strips size pieces
artichoke hearts sixths
pine nuts skillet heat
pesto homemade variety
chocolate candies e.g tm
chocolate layer
package potato cheese pierogies
fillets cubes
package fideo pasta
dogs pieces
ground seeds
castor sugar superfine sugar
package baby arugula
cereal rings
morton® kosher salt
brussels sprouts lengthwise
package honey ham
fish chunks
tonkatsu sauce barbeque
package pasta choice shape
packages mushrooms
fraiche
style bread
onion juice
boneless half
container spread
dashes bitters
packages pizza crust
capacola sausage
sweet potatoes fry size pieces
coarse sea salt pepper
package cut asparagus

boneless leg lamb
kitchen twine
rotisserie chicken bones
zucchini shape
chicken livers size pieces
mushrooms variety choice
split skin chicken breasts bones fat poultry scissors
zucchini lengthwise crosswise inch pieces
package bread cubes
loaves bread
cans tomatoes salt
wine juice
herbs thyme basil
firm fish grouper tilapia tuna chunks
ferns
roast coffee
strawberries chunks
sirloin steaks inch thick
thai chile peppers
wax peppers
package crumbs
onion medium dice
cartons yogurt
gingersnaps
jar olives liquid
beef eye round strips
smoke concentrate
package crispy potato rounds tater tots®
package food velveeta®
sugar pumpkin inch dice
spring mix salad greens
oval slices
sugar maple
jar tomato basil pasta sauce
nut bran cereal
% parmesan
skinless boneless chicken cutlets inch thickness
links slices
milk chocolate candy
log cheese chevre
package sausage links
frozen cherries
style yogurt fage®
lasagna noodles pieces
fluid stout beer guinness®
package baby potatoes
tomatoes onion celery
but

doughnuts
package crawfish tails
cheddar cheese cheddar
color food coloring
baking potatoes inch pieces
sheet nori seaweed
ham dice
regular
bosc pear pieces
pear pieces
slices firm
packages buttermilk biscuits varieties
mushrooms dice
bratwurst inch pieces
calorie sweetener splenda
herb garlic cream cheese spread
skin boneless chicken
hearts fourths
package pastry inch pie crust
coffee kahlua®
package chocolate pudding pie
dashi granules
bonito shavings fish flakes
strawberries wash hull layer time potato masher
fruit juice fruit juice concentrate water
sugar pectin
sugar sugar substitute honey taste
ball® pint lids bands
package pizza
package cream cheese chives
buckwheat groats
container blueberries
lime wedge
orange wedge
apple cinnamon granola
grape clusters
fuyu persimmons
carrot dice
cantaloupe balls
inch watermelon
mango inch dice
beef sirloin roast
sauce steak sauce®
package hillshire farm® sausage variety
zucchini ribbons peeler seedy core
chili hormel®
bottle seed salad
packa

package cherry cranberries flavor craisins®
scallop shells
pure
ears corn halves
chili beans brooks®
drops sauce pepper sauce
day bread inch
apples inch pieces
capers brine
quality
strip zest
spicy juice cocktail
serrano ham prosciutto inch dice
calvados apple brandy
fabada beans beans
ham inch cubes
chorizo sausage inch thick
sausage blood sausage inch thick
knipla
puree libby's®
leek tender green parts lengthwise
marzano style plum tomatoes
graham crackers rectangles
head romaine lettuce end
parsnips wedges
skinless boneless chicken breasts chunks
bread triangles chips
spicy juice v8®
soy maggi®
balls mozzarella cheese
liqueur marnier cointreau
sugar orange
box chocolate wafers
nonpareils stars peppermint patties
chocolate kisses foil
package fettuccini pasta
package pork tenderloins
matchstick style carrots
ciabatta bread split half
package roast beef mix
package idahoan® potatoes
chard kale
mayocoba beans
slices ham cubes
inch potato chunks
pot barley
pumpkin puree
fat drippings
li

jar curry paste patak's®
bone chickens giblets
corn tortillas pieces
package fudge brownie mix batter box
slices hickory bacon
pato® salsa fresco
jar picante sauce pace® picante sauce
cans chiles ortega® chiles
style salad wishbone®
oatmeal cooking
vanilla plain soymilk
milliliter proof rum bacardi®
gallons apple cider
cranberry
style tortilla chips scoops®
package lemon mix jell o®
soybean oil oil
plain soymilk
package chocolate cookie
count cans biscuit dough pillsbury grands ®
patron®
maille® dijon mustard
tomatoes inch thick
amarillo peppers
mix jell o®
rose's® grenadine syrup
inch pastry base
skinless fillets
shape
turkey giblets
aji mirin rice wine
boneless beef chuck cubes
container beef stock concentrate knorr® homestyle stock
packet soup mix knorr®
halibut inch thick slices
peppers fry pieces
pineapple sherbet
liter cherry
vodka flavors citron raspberry mandarin
lemon lime wedges
dash rose's® lime juice
liter bottle grape soda
spaghetti noodles halves
slices potato
infusions® 

energy drink bull™
package fruit candies skittles®
milliliter bottle merlot wine
inch orange peel strips
inch lemon peel strips
worms
turkey cutlets chunks
inch flour tortilla strips 'tombstones
tennessee
ground coriander seeds
boneless beef round steak
medium bell strips
cheddar cheese sargento® chef blends® state cheddar® cheese
strips hickory bacon crisp cut
brioche buns split
arthur coconut
king arthur
box uncle rice quinoa pepper
uncle ben's® rice® garden
package breast tyson®
king arthur flour
confectioners sugar glazing sugar
berries jam
calamari tubes tentacles
cream butter
blood orange vinegar
ghosts peeps®
package chocolate almond bark chocolate
peppermint candy canes pieces
fluid pepper sauce frank redhot®
count biscuit dough
root inch dice
egg beaten egg wash
cream vanilla ice cream
styrofoam blocks
corn tortillas inch pieces
inch thick slices baguette bias
garlic herb cheese boursin®
cocoa mix miss®
mustard keen's®
package egg roll wrappers quarters
link chorizo sausage
co

slices originals turkey breast
slices originals picante provolone
dietz watson cranberry honey mustard
package bacon crosswise
dietz grain mustard
grain crackers
haricots verts beans
boneless skinless turkey breast size pieces
slices gourmet
hoagie dressing
harissa pepper paste
hanout spice mix
shallots inch thick
dietz watson stone ground
slices roast pork
dietz watson dill pickles
wheat bran cereal kellogg's® bran™
sodium tomato sauce
blend vegeta®
flour lily®
slices provolone cheese strips
slices fe turkey breast strips
cherry pear tomatoes
dietz watson gourmet
dietz watson chicken bacon
bottle dietz watson sandwich
dietz watson picante provolone cheese
dietz watson peppadew state cheddar cheese
dietz watson cheese
inch okra slices
piece cassia cinnamon pieces
cotto ham
button mushrooms oil
dragon eyes
habanero pepper flakes
honey sweetener
liter orange juice
bears
tart cherry juice concentrate
chipotle spice rub
cabbage chunks
jujube dates
shrimp saewujeot
chestnuts strips
pear mat